In [1]:
import tensorflow as tf
tf.random.set_seed(42)
tf.config.run_functions_eagerly(True)
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import shutil

#from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Activation, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Concatenate, BatchNormalization
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import mobilenet_v2

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
train_triplets_path = '../input/triplets/train_triplets.txt'
test_triplets_path = '../input/triplets/test_triplets.txt'
food_path = '/content/drive/MyDrive/ml_task4/food.zip'
image_path = '../input/food-img/food'

# My Implementation

In [3]:
train_triplets = pd.read_csv(train_triplets_path, delim_whitespace=True, header=None, names =['anchor','positive','negative'], dtype='str')
test_triplets = pd.read_csv(test_triplets_path, delim_whitespace=True, header=None, names =['anchor','positive','negative'], dtype='str')

train_samples, val_samples = train_test_split(train_triplets, test_size=0.2)

In [4]:
target_shape = (224, 224)
IMG_WIDTH = 224
IMG_HEIGHT = 224

def preprocess_image(filename,training=True):
    image_string = tf.io.read_file(image_path + '/' + filename + '.jpg')
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32)
    image = image / 127.5 - 1
    image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH))
    if training:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
    return image

def preprocess_triplets_train(anchor, positive, negative):
    anchor_img = preprocess_image(anchor)
    positive_img = preprocess_image(positive)
    negative_img = preprocess_image(negative)
    
    return tf.stack([anchor_img, positive_img, negative_img], axis=0), 1

def preprocess_triplets_test(anchor, positive, negative):
    anchor_img = preprocess_image(anchor, training=False)
    positive_img = preprocess_image(positive, training=False)
    negative_img = preprocess_image(negative, training=False)
    
    return tf.stack([anchor_img, positive_img, negative_img], axis=0)

def generate_dataset(triplet_df, training=True):
    anchor_images = triplet_df['anchor']
    positive_images = triplet_df['positive']
    negative_images = triplet_df['negative']

    anchor_dataset = tf.data.Dataset.from_tensor_slices(anchor_images)
    positive_dataset = tf.data.Dataset.from_tensor_slices(positive_images)
    negative_dataset = tf.data.Dataset.from_tensor_slices(negative_images)

    dataset = tf.data.Dataset.zip((anchor_dataset, positive_dataset, negative_dataset))
    if training:
        dataset = dataset.map(preprocess_triplets_train,num_parallel_calls=tf.data.experimental.AUTOTUNE)
    else:
        dataset = dataset.map(preprocess_triplets_test,num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset

In [5]:
def create_model():
    base_cnn = tf.keras.applications.MobileNetV2(weights="imagenet", input_shape=target_shape + (3,), include_top=False)
    base_cnn.trainable = False 

#   #flatten = layers.Flatten()(base_cnn.output)
    flatten = tf.keras.layers.GlobalAveragePooling2D()(base_cnn.output)
    dense1 = layers.Dense(128, activation="relu")(flatten)
    output = layers.Lambda(lambda t: tf.math.l2_normalize(t, axis=1))(dense1)
  
    embedding = Model(inputs = base_cnn.input, outputs = output, name="Embedding")
    
    
    inputs = tf.keras.Input(shape=(3, IMG_HEIGHT, IMG_WIDTH, 3))
    anchor, positive, negative = inputs[:, 0, ...], inputs[:, 1, ...], inputs[:, 2, ...]

    anchor_embedding = embedding(anchor)
    positive_embedding = embedding(positive)
    negative_embedding = embedding(negative)

    embeddings = tf.stack([anchor_embedding, positive_embedding, negative_embedding], axis=-1)
    siamese_network = Model(inputs=inputs, outputs=embeddings)
    siamese_network.summary()
    return siamese_network

In [6]:
def compute_distances(embeddings):
  #change this to triplet output in the future
  anchor, positive, negative = embeddings[..., 0], embeddings[..., 1], embeddings[..., 2]
  ap_distance = tf.reduce_sum(tf.square(anchor - positive), 1)
  an_distance = tf.reduce_sum(tf.square(anchor - negative), 1)
  return (ap_distance, an_distance)

def triplet_loss(_, embeddings):
  ap_distance, an_distance = compute_distances(embeddings)
  #might want to change this to L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
  #softplus makes sure distance is positive, smooth approximation of ReLU
  return tf.reduce_mean(tf.math.softplus(ap_distance - an_distance))

def accuracy(_, embeddings):
  ap_distance, an_distance = compute_distances(embeddings)
  # equal to 1 if ap_distance <= an_distance, 0 else, calculates mean along all triplets
  return tf.reduce_mean(
    tf.cast(tf.greater_equal(an_distance, ap_distance), tf.float32))

In [9]:
model = create_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=triplet_loss,
              metrics=[accuracy])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 3, 224, 224, 0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_3 (Sli (None, 224, 224, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_4 (Sli (None, 224, 224, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_5 (Sli (None, 224, 224, 3)  0           input_4[0][0]                    
____________________________________________________________________________________________

In [ ]:
train_dataset = generate_dataset(train_samples)
val_dataset = generate_dataset(val_samples)
train_image_count = train_samples.shape[0]

train_dataset = train_dataset.shuffle(1024, reshuffle_each_iteration=True).repeat().batch(32)
train_dataset = train_dataset.prefetch(8)

val_dataset = val_dataset.batch(32)
val_dataset = val_dataset.prefetch(8)

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
history = model.fit(train_dataset, steps_per_epoch=train_image_count // 32,epochs=4,validation_data=val_dataset,validation_steps=10,callbacks=[cp_callback])

In [10]:
model.load_weights('../input/4epochs/cp.ckpt')

In [ ]:
import shutil
shutil.make_archive(checkpoint_dir, 'zip', checkpoint_dir)

In [11]:
def create_inference_model(model):
    ap_distance, an_distance = compute_distances(model.output)
    predictions = tf.cast(tf.greater_equal(an_distance, ap_distance), tf.int8)
    return tf.keras.Model(inputs=model.inputs, outputs=predictions)

In [12]:
inference_model = create_inference_model(model)

In [13]:
test_dataset = generate_dataset(test_triplets, training=False).batch(256).prefetch(2)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


In [14]:
 predictions = inference_model.predict(
        test_dataset,
        verbose=1)

233/233 [==============================] - 461s 2s/step


In [15]:
print(predictions)

[0 0 0 ... 1 1 0]


In [16]:
# Create submission file

np.savetxt('submission.txt', predictions, fmt='%d')